In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def guided_filted(normI, corrected_t, w, eps):
    M, N, _ = normI.shape
    padded = np.pad(normI, ((int(w/2), int(w/2)), (int(w/2), int(w/2)), (0, 0)), 'edge')
    padded_t = np.pad(corrected_t, ((int(w/2), int(w/2)), (int(w/2), int(w/2))), 'edge')   
    refined_t = np.zeros((M, N))
    for i, j in np.ndindex(refined_t.shape):
        I = padded[i:i + w, j:j + w, :]
        t = padded_t[i:i + w, j:j + w]
        mean_I = cv.boxFilter(I, -1, (w, w), normalize=True)
        mean_t = cv.boxFilter(t, -1, (w, w), normalize=True)
        corr_I = cv.boxFilter(I * I, -1, (w, w), normalize=True)
        corr_I_t = cv.boxFilter(I * t, -1, (w, w), normalize=True)
        var_I = corr_I - mean_I * mean_I
        cov_I_t = corr_I_t - mean_I * mean_t
        a = cov_I_t / (var_I + eps)
        b = mean_t - a * mean_I
        mean_a = cv.boxFilter(a, -1, (w, w), normalize=True)
        mean_b = cv.boxFilter(b, -1, (w, w), normalize=True)
        refined_t[i, j] = mean_a * normI[i, j, 0] + mean_b
    return refined_t

In [ ]:
I = cv.imread('test.jpg')
I = cv.cvtColor(I, cv.COLOR_BGR2RGB)
k = cv.boxFilter(I, -1, (5, 5))